In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [9]:
df_customer_flight= pd.read_csv("file/Customer Flight Activity.csv", index_col=0)
df_customer_flight.head()

,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
Loyalty Number,,,,,,,,,
100018,2017,1,3,0,3,1521,152.0,0,0
100102,2017,1,10,4,14,2030,203.0,0,0
100140,2017,1,6,0,6,1200,120.0,0,0
100214,2017,1,0,0,0,0,0.0,0,0
100272,2017,1,0,0,0,0,0.0,0,0


## Los comentarios respecto a la observación de esta tabla se encuentran dentro del archivo llamado documentación.md 
## Ahora empezamos a hacer las exploraciones de la tabla

In [11]:
df_customer_flight.shape


(405624, 9)

In [12]:
df_customer_flight.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405624 entries, 100018 to 999986
Data columns (total 9 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Year                         405624 non-null  int64  
 1   Month                        405624 non-null  int64  
 2   Flights Booked               405624 non-null  int64  
 3   Flights with Companions      405624 non-null  int64  
 4   Total Flights                405624 non-null  int64  
 5   Distance                     405624 non-null  int64  
 6   Points Accumulated           405624 non-null  float64
 7   Points Redeemed              405624 non-null  int64  
 8   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 30.9 MB


In [42]:
# Saco el resumen estadistico de las columnas numericas
df_customer_flight.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,405624.0,2017.500000,0.500001,2017.0,2017.00,2017.5,2018.00,2018.0
Month,405624.0,6.500000,3.452057,1.0,3.75,6.5,9.25,12.0
Flights Booked,405624.0,4.115052,5.225518,0.0,0.00,1.0,8.00,21.0
Flights with Companions,405624.0,1.031805,2.076869,0.0,0.00,0.0,1.00,11.0
Total Flights,405624.0,5.146858,6.521227,0.0,0.00,1.0,10.00,32.0
Distance,405624.0,1208.880059,1433.155320,0.0,0.00,488.0,2336.00,6293.0
Points Accumulated,405624.0,123.692721,146.599831,0.0,0.00,50.0,239.00,676.5
Points Redeemed,405624.0,30.696872,125.486049,0.0,0.00,0.0,0.00,876.0
Dollar Cost Points Redeemed,405624.0,2.484503,10.150038,0.0,0.00,0.0,0.00,71.0


In [14]:
# busco los nulos
df_customer_flight.isna().sum()/df_customer_flight.shape[0]*100

Year                           0.0
Month                          0.0
Flights Booked                 0.0
Flights with Companions        0.0
Total Flights                  0.0
Distance                       0.0
Points Accumulated             0.0
Points Redeemed                0.0
Dollar Cost Points Redeemed    0.0
dtype: float64

In [ ]:
# Reviso si existen valores duplicados
df_customer_flight.duplicated()

Loyalty Number
100018    False
100102    False
100140    False
100214    False
100272     True
          ...  
999902     True
999911     True
999940    False
999982     True
999986     True
Length: 405624, dtype: bool

In [20]:
df_customer_flight.duplicated().sum()

np.int64(228705)

In [23]:
# LLamo la otra base de datos 
df_customer_loyalty= pd.read_csv("file/Customer Loyalty History.csv", index_col=0)
df_customer_loyalty.head()

,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
Loyalty Number,,,,,,,,,,,,,,,
480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [24]:
df_customer_loyalty.shape

(16737, 15)

In [25]:
df_customer_loyalty.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16737 entries, 480934 to 652627
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             16737 non-null  object 
 1   Province            16737 non-null  object 
 2   City                16737 non-null  object 
 3   Postal Code         16737 non-null  object 
 4   Gender              16737 non-null  object 
 5   Education           16737 non-null  object 
 6   Salary              12499 non-null  float64
 7   Marital Status      16737 non-null  object 
 8   Loyalty Card        16737 non-null  object 
 9   CLV                 16737 non-null  float64
 10  Enrollment Type     16737 non-null  object 
 11  Enrollment Year     16737 non-null  int64  
 12  Enrollment Month    16737 non-null  int64  
 13  Cancellation Year   2067 non-null   float64
 14  Cancellation Month  2067 non-null   float64
dtypes: float64(4), int64(2), object(9)
memory usage: 2.0

In [38]:
# Saco el resumen estadistico de la columnas numericas
df_customer_loyalty.describe(include=object).T

,count,unique,top,freq
Country,16737,1,Canada,16737
Province,16737,11,Ontario,5404
City,16737,29,Toronto,3351
Postal Code,16737,55,V6E 3D9,911
Gender,16737,2,Female,8410
Education,16737,5,Bachelor,10475
Marital Status,16737,3,Married,9735
Loyalty Card,16737,3,Star,7637
Enrollment Type,16737,2,Standard,15766


In [28]:
# busco los nulos
df_customer_loyalty.isna().sum()/df_customer_loyalty.shape[0]*100

Country                0.000000
Province               0.000000
City                   0.000000
Postal Code            0.000000
Gender                 0.000000
Education              0.000000
Salary                25.321145
Marital Status         0.000000
Loyalty Card           0.000000
CLV                    0.000000
Enrollment Type        0.000000
Enrollment Year        0.000000
Enrollment Month       0.000000
Cancellation Year     87.650117
Cancellation Month    87.650117
dtype: float64

In [31]:
# Reviso si existen valores duplicados
df_customer_loyalty.duplicated()

Loyalty Number
480934    False
549612    False
429460    False
608370    False
530508    False
          ...  
823768    False
680886    False
776187    False
906428    False
652627    False
Length: 16737, dtype: bool

In [32]:
df_customer_loyalty.duplicated().sum()

np.int64(0)

In [48]:
## union de DF, utilizo el método Merge porque me permite unirlos por una columna comun que ya he identificado (loyalty number)
df_customer = df_customer_flight.merge(df_customer_loyalty, on=["Loyalty Number"], how="right")
df_customer.head()

,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
Loyalty Number,,,,,,,,,,,,,,,,,,,,,,,,
480934,2017,1,0,0,0,0,0.0,0,0,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
480934,2017,2,3,0,3,2823,282.0,0,0,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
480934,2017,3,0,0,0,0,0.0,0,0,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
480934,2017,4,0,0,0,0,0.0,0,0,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
480934,2018,10,6,2,8,3352,335.0,465,38,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN


In [ ]:
# confirmo que se han unido
df_customer.shape

(405624, 24)

In [55]:
# En el estudio exploratorio de las 2 df duplicadas he observados filas duplicadas y valores nulos, ahora gestiono esto de manera conjunta en todo el DF
#Primero, veo cuantas filas duplicadas hay 
duplicadas = df_customer.duplicated().sum()
print(f"Total de filas duplicadas: {duplicadas}")

Total de filas duplicadas: 1864


In [58]:
#Ver solo las filas duplicadas (exceptuando la primera ocurrencia)
duplicadas = df_customer[df_customer.duplicated()]
print(df_duplicadas)

                Year  Month  Flights Booked  Flights with Companions  \
Loyalty Number                                                         
100272          2017      1               0                        0   
100301          2017      1               0                        0   
100364          2017      1               0                        0   
100380          2017      1               0                        0   
100504          2017      1               0                        0   
...              ...    ...             ...                      ...   
999891          2018     12               0                        0   
999902          2018     12               0                        0   
999911          2018     12               0                        0   
999982          2018     12               0                        0   
999986          2018     12               0                        0   

                Total Flights  Distance  Points Accumulated  Po

In [59]:
#Veo las filas fuplicadas incluyendo la primera 
df_duplicadas_todas = duplicadas[duplicadas.duplicated(keep=False)]
print(df_duplicadas_todas)


                Year  Month  Flights Booked  Flights with Companions  \
Loyalty Number                                                         
678205          2017      1               0                        0   
678205          2017      1               0                        0   
678205          2017      2               0                        0   
678205          2017      2               0                        0   
678205          2017      3               0                        0   
678205          2017      3               0                        0   
678205          2017      4               0                        0   
678205          2017      4               0                        0   
678205          2017      5               0                        0   
678205          2017      5               0                        0   
678205          2017      6               0                        0   
678205          2017      6               0                     

In [60]:
#Elimino los duplicados
df_sin_duplicados = duplicadas.drop_duplicates()

In [71]:
# ahora gestiono mis nulos.
# Convertimos los nulos de 'Cancellation Year' y 'Cancellation Month' a 0 :porque los clientes activos no tienen año de cancelación y 'Activo' para identificar clientes que siguen en el programa.
#Creo por asignación directa una nueva columna “Loyalty Status”
# Activo si el valor en “Cancellation Month” es NaN
# Cancelled si no es NaN
df_sin_duplicados["Cancelattion"] = df_sin_duplicados["Cancellation Month"].apply( lambda dato: "Active" if pd.isna(dato) else "Cancelled")


In [72]:
df_sin_duplicados.head()

,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month,Cancelattion
Loyalty Number,,,,,,,,,,,,,,,,,,,,,,,,,
193662,2017,1,0,0,0,0,0.0,0,0,Canada,Yukon,Whitehorse,Y2K 6R0,Male,Bachelor,51124.0,Married,Star,3844.57,Standard,2012,5,NaN,NaN,Active
193662,2018,8,0,0,0,0,0.0,0,0,Canada,Yukon,Whitehorse,Y2K 6R0,Male,Bachelor,51124.0,Married,Star,3844.57,Standard,2012,5,NaN,NaN,Active
193662,2018,11,0,0,0,0,0.0,0,0,Canada,Yukon,Whitehorse,Y2K 6R0,Male,Bachelor,51124.0,Married,Star,3844.57,Standard,2012,5,NaN,NaN,Active
746226,2017,1,0,0,0,0,0.0,0,0,Canada,British Columbia,Whistler,V6T 1Y8,Female,Bachelor,63501.0,Married,Star,4089.04,Standard,2018,1,NaN,NaN,Active
746226,2017,2,0,0,0,0,0.0,0,0,Canada,British Columbia,Whistler,V6T 1Y8,Female,Bachelor,63501.0,Married,Star,4089.04,Standard,2018,1,NaN,NaN,Active


In [73]:
# Reemplazar los nulos en la columna "Salario" con la mediana
#fillna() reemplaza los NaN por el valor que le pases.#median() calcula la mediana de la columna "Salario"# inplace=True modifica el DataFrame directamente (sin necesidad de reasignarlo).
df_sin_duplicados["Salary"].fillna(df_sin_duplicados["Salary"].median(), inplace=True)

In [74]:
df_sin_duplicados['Salary'].isnull().sum()

np.int64(0)